# Stirling Measure: Parameter Estimation

This notebook demonstrates how to estimate parameters (a,b) from observed data using the Stirling Measure.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import sys
import os

# Add the project root to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import our generalized Stirling implementation
from src.core.python.stirling import GeneralizedStirling

## 1. Generating Synthetic Data

First, let's generate some synthetic data with known parameters a=0.5 and b=1.2

In [ ]:
# Set true parameters
true_a = 0.5
true_b = 1.2

# Create a generalized Stirling calculator with these parameters
gs = GeneralizedStirling(a=true_a, b=true_b)

# Generate (n,k) pairs
n_values = range(5, 20)
k_values = range(2, 8)

data = []
for n in n_values:
    for k in k_values:
        if k <= n:  # Ensure k <= n
            # Calculate S_{n,k}, S_{n+1,k}, and S_{n,k-1}
            s_n_k = gs.compute(n, k)
            s_n_plus_1_k = gs.compute(n+1, k)
            s_n_k_minus_1 = gs.compute(n, k-1) if k > 1 else 0
            
            # Calculate the Stirling measure with some noise
            if s_n_k > 0:  # Avoid division by zero
                stirling_measure = (s_n_plus_1_k - s_n_k_minus_1) / s_n_k
                # Add some noise to make it realistic
                noisy_measure = stirling_measure + np.random.normal(0, 0.2)
                
                data.append({
                    'n': n,
                    'k': k,
                    'S_n_k': s_n_k,
                    'S_n_plus_1_k': s_n_plus_1_k,
                    'S_n_k_minus_1': s_n_k_minus_1,
                    'measure': noisy_measure,
                    'true_measure': true_a * n + true_b * k
                })

# Convert to DataFrame
df = pd.DataFrame(data)
df.head()

## 2. Estimating Parameters using Linear Regression

Now, let's use linear regression to estimate parameters a and b from our data.

In [ ]:
# Prepare data for regression
X = df[['n', 'k']].values
y = df['measure'].values

# Fit linear regression model
model = LinearRegression()
model.fit(X, y)

# Extract estimated parameters
estimated_a = model.coef_[0]
estimated_b = model.coef_[1]

print(f"True parameters: a={true_a}, b={true_b}")
print(f"Estimated parameters: a={estimated_a:.4f}, b={estimated_b:.4f}")

## 3. Visualizing the Results

Let's visualize how well our estimated parameters match the true values.

In [ ]:
# Calculate predicted values
df['predicted_measure'] = estimated_a * df['n'] + estimated_b * df['k']

# Create the plot
plt.figure(figsize=(12, 6))

# Create 3D scatter plot
ax = plt.axes(projection='3d')
ax.scatter3D(df['n'], df['k'], df['measure'], c='blue', label='Observed')
ax.scatter3D(df['n'], df['k'], df['true_measure'], c='green', alpha=0.5, label='True')
ax.scatter3D(df['n'], df['k'], df['predicted_measure'], c='red', alpha=0.5, label='Predicted')

ax.set_xlabel('n (number of elements)')
ax.set_ylabel('k (number of groups)')
ax.set_zlabel('Stirling Measure')
ax.set_title('Stirling Measure: True vs. Estimated')
plt.legend()
plt.show()

## 4. Using the Estimated Parameters for Predictions

Now let's see how our estimated parameters perform in predicting future clustering behavior.

In [ ]:
# Create a new generalized Stirling calculator with estimated parameters
gs_estimated = GeneralizedStirling(a=estimated_a, b=estimated_b)

# Compare values for a few test cases
test_cases = [(25, 5), (30, 8), (40, 10)]
results = []

for n, k in test_cases:
    true_value = gs.compute(n, k)
    estimated_value = gs_estimated.compute(n, k)
    error_pct = abs((estimated_value - true_value) / true_value * 100) if true_value != 0 else float('inf')
    
    results.append({
        'n': n,
        'k': k,
        'true_value': true_value,
        'estimated_value': estimated_value,
        'error_pct': error_pct
    })

pd.DataFrame(results)

## 5. Real-World Application: Route Optimization

In a real-world scenario, we would:
1. Observe how deliveries naturally cluster into routes
2. Calculate the Stirling measure for each observation
3. Estimate parameters a and b
4. Use these parameters to make optimal decisions about route assignments

For example, when deciding whether to assign a new delivery to an existing route or create a new route, we would compare:
- Cost of adding to existing route: (a*n + b*k) * S(n,k)
- Cost of creating new route: S(n,k-1)

This provides a mathematically sound basis for making routing decisions that align with the natural tendencies of our specific delivery environment.